In [1]:
# =================================================================================
# 💎 CULTUREVERSE FINAL: ROBUST KAGGLE EDITION
# =================================================================================
# ✅ 1 Epoch = 781 Steps (Optimized for 0.16 Loss)
# ✅ Strict "6-Heading" Output Format
# ✅ Robust Hybrid System for New Idioms
# =================================================================================

import os
import sys
import warnings
import gc
import shutil
import pickle
import numpy as np

# --- 1. AGGRESSIVE DEPENDENCY FIX (CRITICAL - DO NOT TOUCH) ---
print("🛠️ CLEANING ENVIRONMENT & INSTALLING STACK...")

# 1. Force uninstall conflicting libraries
os.system("pip uninstall -y pyarrow")

# 2. Install compatible PyArrow (Fixes Kaggle binary conflict)
os.system("pip install -q -U \"pyarrow>=15.0.0\"")

# 3. Install Stack (Pinned TRL 0.12.0 for stability)
os.system("pip install -q -U bitsandbytes transformers peft accelerate")
os.system("pip install -q -U datasets") 
os.system("pip install -q -U trl==0.12.0")  # Strictly pinned
os.system("pip install -q -U sentence-transformers faiss-cpu requests")

print("✅ Installation Complete. Importing libraries...")

# --- 2. IMPORTS & SETUP ---
import torch
import pandas as pd
import bitsandbytes as bnb
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from trl import SFTTrainer, SFTConfig
from sentence_transformers import SentenceTransformer
import faiss

# --- 2.1 HF LOGIN ---
try:
    from kaggle_secrets import UserSecretsClient
    from huggingface_hub import login
    print("🔑 Logging in to Hugging Face...")
    user_secrets = UserSecretsClient()
    hf_token = user_secrets.get_secret("HF_TOKEN")
    login(token=hf_token)
    print("✅ Logged in successfully.")
except Exception as e:
    print(f"⚠️ HF Login Skipped: {e} (Continuing anyway...)")

warnings.filterwarnings("ignore")

# Define Paths
PROJECT_ROOT = "/kaggle/working/CultureVerse_Final"
MODEL_DIR = f"{PROJECT_ROOT}/model_adapter"
VECTOR_DIR = f"{PROJECT_ROOT}/vector_db"
DEPLOYMENT_DIR = f"{PROJECT_ROOT}/deployment"

for d in [PROJECT_ROOT, MODEL_DIR, VECTOR_DIR, DEPLOYMENT_DIR]: 
    os.makedirs(d, exist_ok=True)

# Find Dataset Automatically
DATA_FILE = None
for root, dirs, files in os.walk("/kaggle/input"):
    for file in files:
        if file.endswith(".csv") and "cultureverse" in file.lower():
            DATA_FILE = os.path.join(root, file)
            print(f"📍 Found Data: {DATA_FILE}")
            break
    if DATA_FILE: break

if not DATA_FILE: raise FileNotFoundError("❌ CSV Not Found! Add data to input.")

# =================================================================================
# 🟢 PHASE 1: FINE-TUNING (ROBUST 1 EPOCH CONFIG)
# =================================================================================
print(f"\n{'='*40}")
print("🧠 PHASE 1: TRAINING (Robust 1 Epoch -> ~781 Steps)")
print(f"{'='*40}")

# 1. Strict Output Formatter
def format_prompt(example):
    """
    Forces the model to output strictly in the requested 6-heading format.
    """
    def generate_text(p, l, c, cmp, t, m, ex, tn):
        has_example = isinstance(ex, str) and len(str(ex).strip()) > 5
        
        # System instruction: Defines the persona and the REQUIRED format
        sys_msg = (
            "You are a cultural expert. Analyze the input phrase and provide a detailed structured response "
            "using EXACTLY these headings:\n"
            "Meaning, Cultural Origin, Usage Context, Emotional Tone, Example, Cultural Tag."
        )
        
        if not has_example:
            sys_msg += " If no example is provided, GENERATE a realistic, culturally appropriate scenario."
        
        # Mapping CSV columns to your specific headings
        # Phrase -> Phrase
        # Meaning -> Meaning
        # Language -> Cultural Origin
        # Category/Complexity -> Usage Context
        # Tone -> Emotional Tone
        # Example -> Example
        # Tags -> Cultural Tag
        
        return (f"<|im_start|>system\n{sys_msg}<|im_end|>\n"
                f"<|im_start|>user\n"
                f"Phrase: '{p}'\n"
                f"<|im_end|>\n"
                f"<|im_start|>assistant\n"
                f"**Meaning:** {m}\n"
                f"**Cultural Origin:** {l}\n"
                f"**Usage Context:** {c} ({cmp})\n"
                f"**Emotional Tone:** {tn}\n"
                f"**Example:** {ex if has_example else '[GENERATE SCENARIO]'}\n"
                f"**Cultural Tag:** {t}<|im_end|>\n")

    if isinstance(example['phrase'], list):
        return {"text": [generate_text(
            example['phrase'][i], example['language'][i], example['category'][i],
            example['complexity'][i], example['tags'][i], example['meaning'][i],
            example['example'][i], example['tone'][i]
        ) for i in range(len(example['phrase']))]}
    
    return {"text": generate_text(
        example['phrase'], example['language'], example['category'],
        example['complexity'], example['tags'], example['meaning'],
        example['example'], example['tone']
    )}

# 2. Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-7B-Instruct", trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# 3. Load Model (Dual GPU Optimized)
print("🤖 Loading Qwen 2.5 on Dual GPUs...")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-7B-Instruct",
    quantization_config=bnb_config,
    device_map="auto",  # Splits model across T4 x 2
    trust_remote_code=True,
    torch_dtype=torch.float16
)
model.config.use_cache = False
model = prepare_model_for_kbit_training(model)

peft_config = LoraConfig(
    r=64, lora_alpha=16, lora_dropout=0.05, bias="none", 
    task_type="CAUSAL_LM", 
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
)
model = get_peft_model(model, peft_config)

# 4. Training Arguments (ROBUST 1 EPOCH CONFIG)
training_args = SFTConfig(
    output_dir=MODEL_DIR,
    num_train_epochs=1,                 # Exactly 1 Epoch
    per_device_train_batch_size=4,      # 4 per GPU * 2 GPUs = 8
    gradient_accumulation_steps=4,      # 8 * 4 = 32 Effective Batch (vs 64 before). 
                                        # This doubles the steps to ~781, ensuring deep learning.
    learning_rate=2e-4,                 # Robust LR for 700+ steps
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,
    fp16=True,
    logging_steps=25,
    optim="paged_adamw_8bit",
    report_to="none",
    save_strategy="steps",
    save_steps=200,
    save_total_limit=2,
    group_by_length=True,
    max_seq_length=512,
    dataset_text_field="text",
    packing=False
)

# 5. Train
dataset = load_dataset("csv", data_files=DATA_FILE, split="train")
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset.map(format_prompt, batched=True),
    peft_config=peft_config,
    tokenizer=tokenizer,
    args=training_args
)

print(f"🚀 Starting Training (~{len(dataset)//32} Steps | Robust 1 Epoch)...")
trainer.train()

print("💾 Saving Model...")
trainer.model.save_pretrained(MODEL_DIR)
tokenizer.save_pretrained(MODEL_DIR)

# Cleanup
del model, trainer
torch.cuda.empty_cache()
gc.collect()
print("✅ Phase 1 Complete.")

# =================================================================================
# 🟡 PHASE 2: DEEP VECTOR DATABASE (ALL 8 COLUMNS)
# =================================================================================
print(f"\n{'='*40}")
print("📚 PHASE 2: BUILDING DEEP VECTOR DB")
print(f"{'='*40}")

df = pd.read_csv(DATA_FILE).fillna("")

rich_texts = []
print("🔨 Addressing vectors to ALL words in ALL 8 columns...")
for _, r in df.iterrows():
    text = (f"Phrase: {r['phrase']} | Language: {r['language']} | "
            f"Meaning: {r['meaning']} | Situation: {r['example']} | "
            f"Tags: {r['tags']} | Category: {r['category']} | "
            f"Tone: {r['tone']} | Complexity: {r['complexity']}")
    rich_texts.append(text)

embedder = SentenceTransformer('all-MiniLM-L6-v2', device='cuda')
embeddings = embedder.encode(rich_texts, convert_to_numpy=True, show_progress_bar=True, batch_size=64)

index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

faiss.write_index(index, f"{VECTOR_DIR}/idioms.index")
with open(f"{VECTOR_DIR}/metadata.pkl", "wb") as f:
    pickle.dump(df.to_dict('records'), f)

print("✅ Phase 2 Complete.")

# =================================================================================
# 🔵 PHASE 3: HYBRID SYSTEM CREATION
# =================================================================================
print(f"\n{'='*40}")
print("⚙️ PHASE 3: CREATING HYBRID SYSTEM")
print(f"{'='*40}")

hybrid_system_code = '''"""
Hybrid Idiom Explainer - Production System
"""
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
from sentence_transformers import SentenceTransformer
import faiss
import pickle
import requests
from typing import Dict
from functools import lru_cache

class HybridIdiomExplainer:
    def __init__(self, model_dir: str, vector_db_dir: str, use_gpu: bool = True):
        self.device = "cuda" if use_gpu and torch.cuda.is_available() else "cpu"
        print(f"Loading system on {self.device}...")
        
        # Tier 2: Vector DB
        self.embedder = SentenceTransformer('all-MiniLM-L6-v2')
        self.index = faiss.read_index(f"{vector_db_dir}/idioms.index")
        with open(f"{vector_db_dir}/metadata.pkl", 'rb') as f:
            self.metadata = pickle.load(f)
            
        # Tier 1: Fine-Tuned Model
        self.tokenizer = AutoTokenizer.from_pretrained(model_dir)
        base_model = AutoModelForCausalLM.from_pretrained(
            "Qwen/Qwen2.5-7B-Instruct",
            torch_dtype=torch.float16 if use_gpu else torch.float32,
            device_map="auto"
        )
        self.model = PeftModel.from_pretrained(base_model, model_dir)
        self.model.eval()
        print(f"✅ System ready ({len(self.metadata)} idioms indexed)")
    
    def _tier1_model(self, phrase: str, language: str) -> Dict:
        """Tier 1: Generative Model"""
        # Strict Format Prompt for Inference
        prompt = (f"<|im_start|>system\\n"
                  f"You are a cultural expert. Analyze the phrase and output the result strictly using these headings:\\n"
                  f"Meaning, Cultural Origin, Usage Context, Emotional Tone, Example, Cultural Tag.\\n"
                  f"<|im_end|>\\n"
                  f"<|im_start|>user\\n"
                  f"Phrase: '{phrase}'\\n"
                  f"<|im_end|>\\n<|im_start|>assistant\\n")
        
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.device)
        with torch.no_grad():
            outputs = self.model.generate(
                **inputs, max_new_tokens=400, temperature=0.7, top_p=0.9, do_sample=True
            )
        response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        if "<|im_start|>assistant" in response:
            response = response.split("<|im_start|>assistant")[1].strip()
            
        return {"response": response, "confidence": 0.95, "tier": "fine-tuned-model"}
    
    def _tier2_vector(self, phrase: str) -> Dict:
        """Tier 2: Vector Search"""
        query_embedding = self.embedder.encode([phrase])
        distances, indices = self.index.search(query_embedding.astype('float32'), 1)
        best_idx = indices[0][0]
        if distances[0][0] < 10.0:
            similar = self.metadata[best_idx]
            response = (f"**Meaning:** {similar['meaning']}\\n"
                        f"**Cultural Origin:** {similar['language']}\\n"
                        f"**Usage Context:** {similar['category']}\\n"
                        f"**Emotional Tone:** {similar['tone']}\\n"
                        f"**Example:** {similar['example']}\\n"
                        f"**Cultural Tag:** {similar['tags']}")
            return {"response": response, "confidence": 0.85, "tier": "vector-similarity"}
        return {"response": "", "confidence": 0.0, "tier": "vector-failed"}
    
    def explain(self, phrase: str, language: str="English") -> Dict:
        print(f"\\n🔍 Analyzing: '{phrase}'")
        # 1. Try Model (For creativity & strict format)
        t1 = self._tier1_model(phrase, language)
        if t1['confidence'] > 0.8: return t1
        
        # 2. Try Vector (For new idioms that might match existing ones semantically)
        t2 = self._tier2_vector(phrase)
        if t2['confidence'] > 0.8: return t2
        
        return {"response": "Not Found", "confidence": 0.0}
'''
with open(f"{DEPLOYMENT_DIR}/hybrid_system.py", "w") as f:
    f.write(hybrid_system_code)

print("✅ hybrid_system.py saved.")

# =================================================================================
# 📦 PHASE 4: PACKAGING
# =================================================================================
print(f"\n{'='*40}")
print("📦 PHASE 4: PACKAGING")
print(f"{'='*40}")

readme = """# CultureVerse Hybrid System
## Usage
```python
from deployment.hybrid_system import HybridIdiomExplainer
engine = HybridIdiomExplainer(model_dir="../model_adapter", vector_db_dir="../vector_db")
result = engine.explain("Spill the beans", "English")
print(result['response'])
"""
with open(f"{DEPLOYMENT_DIR}/README.md", "w") as f: 
    f.write(readme)

print("📦 Zipping Final Package...") 
shutil.make_archive("/kaggle/working/CultureVerse_Final", 'zip', PROJECT_ROOT)

print(f"\n🎉 DONE! Output saved to: /kaggle/working/CultureVerse_Final.zip") 
print("👉 Check the 'Output' tab to download the file.")

🛠️ CLEANING ENVIRONMENT & INSTALLING STACK...
Found existing installation: pyarrow 22.0.0
Uninstalling pyarrow-22.0.0:
  Successfully uninstalled pyarrow-22.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 43.9 MB/s eta 0:00:00


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.26.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
cudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
bigframes 2.26.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
pylibcudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 117.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 556.4/556.4 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.3/512.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 861.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 99.5 MB/s eta 0:00:00
✅ Installation Complete. Importing libraries...


2025-12-25 09:45:49.820333: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766655950.206021      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766655950.315116      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766655951.300355      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766655951.300397      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766655951.300400      55 computation_placer.cc:177] computation placer alr

🔑 Logging in to Hugging Face...
✅ Logged in successfully.
📍 Found Data: /kaggle/input/cultureverse-25k-gold-csv/cultureverse_25k_gold.csv

🧠 PHASE 1: TRAINING (Robust 1 Epoch -> ~781 Steps)


...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/391 [00:00<?, ?it/s]

✅ Phase 2 Complete.

⚙️ PHASE 3: CREATING HYBRID SYSTEM
✅ hybrid_system.py saved.

📦 PHASE 4: PACKAGING
📦 Zipping Final Package...

🎉 DONE! Output saved to: /kaggle/working/CultureVerse_Final.zip
👉 Check the 'Output' tab to download the file.


In [2]:
import os
from IPython.display import FileLink

# This generates a clickable download link
print("👇 Click the link below to download your file:")
FileLink(r'CultureVerse_Final.zip')

👇 Click the link below to download your file:


/kaggle/working/CultureVerse_Final.zip